In [1]:
import pandas as pd
import numpy as np
import psycopg2
import math
from datetime import timedelta
import os
%matplotlib inline

con = psycopg2.connect (user = 'postgres',
                password = 'postgres',
                port = '5433',
                host = 'localhost',
                database = 'inicudb')

cur  = con.cursor()

In [2]:
uhidPatient = 'RSHI.0000016373'
#uhidPatient = "RSHI.0000012984"
caseType = 'Death'
#caseType = 'Discharge'
seperator = '_'
path = os.getcwd()
fileName = path+"/Death_Cases/"+uhidPatient+"/"+caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_new_1.csv'

In [3]:
data = pd.read_csv(fileName)

In [4]:
data['series'] = data.hour_series

In [5]:
data.columns

Index(['Unnamed: 0', 'actual_DOB', 'uhid', 'dischargeddate', 'actual_DOA',
       'gender', 'birthweight', 'birthlength', 'birthheadcircumference',
       'inout_patient_status', 'gestationweekbylmp', 'gestationdaysbylmp',
       'baby_type', 'central_temp', 'apgar_onemin', 'apgar_fivemin',
       'apgar_tenmin', 'motherage', 'conception_type', 'mode_of_delivery',
       'steroidname', 'numberofdose', 'dischargestatus', 'birthlength.1',
       'birthheadcircumference.1', 'inout_patient_status.1',
       'gestationweekbylmp.1', 'gestationdaysbylmp.1', 'baby_type.1',
       'central_temp.1', 'apgar_onemin.1', 'apgar_fivemin.1', 'apgar_tenmin.1',
       'motherage.1', 'conception_type.1', 'mode_of_delivery.1',
       'steroidname.1', 'numberofdose.1', 'los', 'day_1', 'hour_series',
       'hour', 'day_x', 'abdomen_girth', 'urine', 'stool', 'stool_passed',
       'entry_timestamp', 'day_y', 'time_divide', 'series'],
      dtype='object')

In [6]:
data.urine


0      NaN
1      NaN
2      NaN
3      NaN
4      NaN
        ..
4093   NaN
4094   NaN
4095   NaN
4096   NaN
4097   NaN
Name: urine, Length: 4098, dtype: float64

In [7]:
data.time_divide.unique()

array([    0.   , 68768.825, 14450.704, 14387.567,  7176.697, 18027.952,
        3553.135, 14407.491, 13670.356, 63726.215, 12640.841,  7167.184])

In [8]:
def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

def split_hour(x):
    try:
        t=str(x)
        return t.split(":")[0]
    except:
        pass
    
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass
    

In [9]:
def urine_check(x):
    try:
        if x > 200:
            return x/10
        else:
            return x
    except:
        pass

In [10]:
def to_float(x):
    if x == '':
        return np.nan
    else:
        return float(x)

In [11]:
dg = data.copy()

In [12]:
dg['urine'] = dg['urine'].apply(to_float)

In [13]:
dg['urine'] = dg['urine'].apply(urine_check)

In [14]:
dg['urine_per_hour'] = dg['urine']/((dg['time_divide'])/(3600))

In [15]:
dt = dg.sort_values(by = ['uhid','hour_series'])

In [16]:
ds = pd.DataFrame()

In [17]:
for i in dt.uhid.unique():
    x = dt[dt['uhid']==i]
    x['urine_per_hour'].fillna(method='bfill',inplace=True)
    ds = ds.append(x,ignore_index=True)
       

In [18]:
cur1 = con.cursor()
cur1.execute("SELECT uhid,creationtime,modificationtime,ph,entrydate FROM apollo.nursing_bloodgas where uhid ='"+ uhidPatient+"'")
cols1 = list(map(lambda x: x[0],cur1.description))
ph = pd.DataFrame(cur1.fetchall(),columns=cols1)

In [19]:
ph

,uhid,creationtime,modificationtime,ph,entrydate
0,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
1,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
2,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
3,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
4,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
5,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
6,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
7,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
8,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
9,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05


In [20]:
def ph_func(x):
    try:
        p = float(x)
        if p>=6.80 and p<=7.8:
            return p
        else:
            return None
    except:
        pass

In [21]:
ph

,uhid,creationtime,modificationtime,ph,entrydate
0,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
1,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
2,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
3,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
4,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
5,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
6,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
7,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
8,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05
9,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05


In [22]:
ph['new_ph'] = ph.ph.apply(ph_func)

In [23]:
ph.dropna(subset=['new_ph'],inplace=True)

In [24]:
ph_df = pd.DataFrame()
for i in ph.uhid.unique():
    x = ph[ph['uhid']==i]
    ph_df = ph_df.append(x,ignore_index=True)

In [25]:
ph

,uhid,creationtime,modificationtime,ph,entrydate,new_ph
0,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
1,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
2,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
3,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
4,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
5,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
6,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
7,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
8,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
9,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13


In [26]:
ph_df

,uhid,creationtime,modificationtime,ph,entrydate,new_ph
0,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
1,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
2,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
3,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
4,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
5,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
6,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
7,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
8,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
9,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13


In [27]:
#ph_df = pd.DataFrame(columns = ph.columns)


In [28]:
ph_df

,uhid,creationtime,modificationtime,ph,entrydate,new_ph
0,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
1,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
2,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
3,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
4,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
5,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
6,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
7,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
8,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13
9,RSHI.0000016373,2019-02-20 00:14:05,2019-02-20 00:14:05,7.13,2019-02-20 00:14:05,7.13


In [29]:
ph_df_1 = None

In [30]:
if len(ph_df.columns)>0:
    ph_df['hour'] = ph_df.entrydate.apply(split_hour)
    ph_df.drop_duplicates(subset=['uhid','hour'],keep='last',inplace=True)
else:
    x = pd.DataFrame(data=[uhidPatient],columns={'uhid'})
    print(x)
    ph_df_1 = pd.DataFrame(columns=ph.columns)
    ph_df_1 = ph_df_1.append(x,ignore_index=True)

In [31]:
if not(ph_df_1 is None) and len(ph_df_1.columns)>0 :
    ph_df_1

In [32]:
type(ds.uhid[0])

str

In [33]:
ds['uhid'] = ds['uhid'].astype(str)


In [34]:
if len(ph_df.columns)>0:
    ss = pd.merge(ds,ph_df,on=['uhid','hour'],how='left')
    ss = ss.drop_duplicates(subset=['uhid','hour_series'],keep='first')
else:
    ss = pd.merge(ds,ph_df_1,on=['uhid'],how='left')
    ss = ss.drop_duplicates(subset=['uhid','hour_series'],keep='first')

In [35]:
ss

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,entry_timestamp,day_y,time_divide,series,urine_per_hour,creationtime,modificationtime,ph,entrydate,new_ph
0,0,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaN,0.0,2019-02-17 14:54:00.000,0.314096,NaT,NaT,NaN,NaT,NaN
1,1,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaN,0.0,2019-02-17 14:55:00.000,0.314096,NaT,NaT,NaN,NaT,NaN
2,2,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaN,0.0,2019-02-17 14:56:00.000,0.314096,NaT,NaT,NaN,NaT,NaN
3,3,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaN,0.0,2019-02-17 14:57:00.000,0.314096,NaT,NaT,NaN,NaT,NaN
4,4,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaN,0.0,2019-02-17 14:58:00.000,0.314096,NaT,NaT,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4093,4093,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,2019-02-20 11:08:54.295000+08:00,2019-02-20,0.0,2019-02-20 11:07:00.000,NaN,NaT,NaT,NaN,NaT,NaN
4094,4094,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,2019-02-20 11:08:54.295000+08:00,2019-02-20,0.0,2019-02-20 11:08:00.000,NaN,NaT,NaT,NaN,NaT,NaN
4095,4095,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,2019-02-20 11:08:54.295000+08:00,2019-02-20,0.0,2019-02-20 11:09:00.000,NaN,NaT,NaT,NaN,NaT,NaN
4096,4096,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,2019-02-20 11:08:54.295000+08:00,2019-02-20,0.0,2019-02-20 11:10:00.000,NaN,NaT,NaT,NaN,NaT,NaN


In [36]:
ss

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,entry_timestamp,day_y,time_divide,series,urine_per_hour,creationtime,modificationtime,ph,entrydate,new_ph
0,0,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaN,0.0,2019-02-17 14:54:00.000,0.314096,NaT,NaT,NaN,NaT,NaN
1,1,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaN,0.0,2019-02-17 14:55:00.000,0.314096,NaT,NaT,NaN,NaT,NaN
2,2,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaN,0.0,2019-02-17 14:56:00.000,0.314096,NaT,NaT,NaN,NaT,NaN
3,3,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaN,0.0,2019-02-17 14:57:00.000,0.314096,NaT,NaT,NaN,NaT,NaN
4,4,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaN,0.0,2019-02-17 14:58:00.000,0.314096,NaT,NaT,NaN,NaT,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4093,4093,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,2019-02-20 11:08:54.295000+08:00,2019-02-20,0.0,2019-02-20 11:07:00.000,NaN,NaT,NaT,NaN,NaT,NaN
4094,4094,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,2019-02-20 11:08:54.295000+08:00,2019-02-20,0.0,2019-02-20 11:08:00.000,NaN,NaT,NaT,NaN,NaT,NaN
4095,4095,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,2019-02-20 11:08:54.295000+08:00,2019-02-20,0.0,2019-02-20 11:09:00.000,NaN,NaT,NaT,NaN,NaT,NaN
4096,4096,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,2019-02-20 11:08:54.295000+08:00,2019-02-20,0.0,2019-02-20 11:10:00.000,NaN,NaT,NaT,NaN,NaT,NaN


In [37]:
len(ss.uhid.unique())

1

In [38]:
cur3 = con.cursor()
cur3.execute("SELECT t1.uhid,t1.entrydate,t1.rbs,t1.skintemp,t1.centraltemp  FROM apollo.nursing_vitalparameters AS t1 where t1.uhid = '"+uhidPatient+"'")
cols3 = list(map(lambda x: x[0], cur3.description))
nv = pd.DataFrame(cur3.fetchall(),columns=cols3)

In [39]:
nv.drop_duplicates(subset=['uhid','entrydate'],keep='first',inplace=True)

In [40]:
nv['hour'] = nv.entrydate.apply(split_hour)

In [41]:
nv.hour.iloc[0]

'2019-02-18 02'

In [42]:
ss.uhid = ss.uhid.astype(str)

In [43]:
ss.hour.iloc[0]

'2019-02-17 14'

In [44]:
s1 = pd.merge(ss,nv,on = ['uhid','hour'], how='left')

In [45]:
len(s1.uhid.unique())

1

In [46]:
s1.drop_duplicates(subset=['uhid','hour_series'],keep='first',inplace=True)

In [47]:
s1

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,urine_per_hour,creationtime,modificationtime,ph,entrydate_x,new_ph,entrydate_y,rbs,skintemp,centraltemp
0,0,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,0.314096,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
1,1,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,0.314096,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
2,2,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,0.314096,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
3,3,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,0.314096,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
4,4,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,0.314096,NaT,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5473,4093,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaT,NaT,NaN,NaT,NaN,2019-02-20 11:00:08.251,NaN,None,36.5
5474,4094,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaT,NaT,NaN,NaT,NaN,2019-02-20 11:00:08.251,NaN,None,36.5
5475,4095,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaT,NaT,NaN,NaT,NaN,2019-02-20 11:00:08.251,NaN,None,36.5
5476,4096,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,NaT,NaT,NaN,NaT,NaN,2019-02-20 11:00:08.251,NaN,None,36.5


In [48]:
def temp_a(x,y):
    if not(x is None) and (x>0):
        return x
    else:
        return y

In [49]:
s1['temp'] = s1.apply(lambda x: temp_a(x['centraltemp'], x['skintemp']), axis=1)

In [50]:
cur2 = con.cursor()
cur2.execute("SELECT t1.uhid,t1.visitdate,t1.visittime,t1.currentdateweight, t1.currentdateheight  FROM apollo.baby_visit AS t1 where t1.uhid = '"+uhidPatient+"'")
cols2 = list(map(lambda x: x[0], cur2.description))
bv = pd.DataFrame(cur2.fetchall(),columns=cols2)

In [51]:
bv

,uhid,visitdate,visittime,currentdateweight,currentdateheight
0,RSHI.0000016373,2019-02-17,17:18:24.308573,2520.0,40.5
1,RSHI.0000016373,2019-02-18,20:16:00,2565.0,NaN
2,RSHI.0000016373,2019-02-19,07:37:00,2583.0,NaN
3,RSHI.0000016373,2019-02-18,06:08:00,2533.0,40.5
4,RSHI.0000016373,2019-02-20,07:00:00,2811.0,NaN


In [52]:
bv.sort_values('visitdate',inplace=True)

In [53]:
def stamp_1(x,y):
    return str(x) +" "+ str(y).split(":")[0]

In [54]:
bv

,uhid,visitdate,visittime,currentdateweight,currentdateheight
0,RSHI.0000016373,2019-02-17,17:18:24.308573,2520.0,40.5
1,RSHI.0000016373,2019-02-18,20:16:00,2565.0,NaN
3,RSHI.0000016373,2019-02-18,06:08:00,2533.0,40.5
2,RSHI.0000016373,2019-02-19,07:37:00,2583.0,NaN
4,RSHI.0000016373,2019-02-20,07:00:00,2811.0,NaN


In [55]:
bv['hour'] = bv.apply(lambda x: stamp_1(x['visitdate'], x['visittime']), axis=1)

In [56]:
def to_str(x):
    return (str(x))

In [57]:
bv['day'] = bv['visitdate'].apply(to_str)

In [58]:
bv.drop_duplicates(subset=['uhid','visitdate'],keep='last',inplace=True)

In [59]:
s1['day'] = s1['hour_series'].apply(split_date_1)

In [60]:
s1.day.unique()

array(['2019-02-17', '2019-02-18', '2019-02-19', '2019-02-20'],
      dtype=object)

In [61]:
s1

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,modificationtime,ph,entrydate_x,new_ph,entrydate_y,rbs,skintemp,centraltemp,temp,day
0,0,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2019-02-17
1,1,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2019-02-17
2,2,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2019-02-17
3,3,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2019-02-17
4,4,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaT,NaN,NaT,NaN,NaT,NaN,None,NaN,NaN,2019-02-17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5473,4093,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaT,NaN,NaT,NaN,2019-02-20 11:00:08.251,NaN,None,36.5,36.5,2019-02-20
5474,4094,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaT,NaN,NaT,NaN,2019-02-20 11:00:08.251,NaN,None,36.5,36.5,2019-02-20
5475,4095,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaT,NaN,NaT,NaN,2019-02-20 11:00:08.251,NaN,None,36.5,36.5,2019-02-20
5476,4096,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaT,NaN,NaT,NaN,2019-02-20 11:00:08.251,NaN,None,36.5,36.5,2019-02-20


In [62]:
s2 = pd.merge(s1,bv,on = ['uhid','day'], how='left')

In [63]:
len(s2.uhid.unique())

1

In [64]:
#Meds



In [65]:
cur7= con.cursor()
cur7.execute("SELECT t1.uhid,t1.startdate,t1.medicineorderdate,t1.medicinename,t1.medicationtype,t2.typevalue FROM apollo.baby_prescription AS t1 LEFT JOIN apollo.ref_medtype AS t2 ON t1.medicationtype = t2.typeid WHERE (t1.medicationtype = 'TYPE0001' OR t1.medicationtype = 'TYPE0009' OR t1.medicationtype = 'TYPE0004') and (t1.uhid ='"+uhidPatient+"')")
cols7 = list(map(lambda x: x[0], cur7.description))
meds = pd.DataFrame(cur7.fetchall(),columns=cols7)

In [66]:
meds

,uhid,startdate,medicineorderdate,medicinename,medicationtype,typevalue
0,RSHI.0000016373,2019-02-19 17:51:20.359000+08:00,2019-02-19 17:51:20.359000+08:00,Colisitin,TYPE0001,Antibiotics
1,RSHI.0000016373,2019-02-19 17:51:20.359000+08:00,2019-02-19 17:51:20.359000+08:00,Meropenem,TYPE0001,Antibiotics
2,RSHI.0000016373,2019-02-18 13:13:01.299000+08:00,2019-02-18 13:13:01.299000+08:00,Piperacilin Tazobactam,TYPE0001,Antibiotics
3,RSHI.0000016373,2019-02-18 13:13:01.299000+08:00,2019-02-18 13:13:01.299000+08:00,Amikacin,TYPE0001,Antibiotics
4,RSHI.0000016373,2019-02-19 21:41:28.752000+08:00,2019-02-19 21:41:28.752000+08:00,Vancomycin,TYPE0001,Antibiotics
5,RSHI.0000016373,2019-02-19 18:02:15.503000+08:00,2019-02-19 18:02:15.503000+08:00,Dopamine,TYPE0004,Inotropes
6,RSHI.0000016373,2019-02-19 18:02:15.503000+08:00,2019-02-19 18:02:15.503000+08:00,Epinephrine,TYPE0004,Inotropes
7,RSHI.0000016373,2019-02-20 00:59:10.504000+08:00,2019-02-20 00:59:10.504000+08:00,Dobutamine,TYPE0004,Inotropes


In [67]:
meds_dummies = pd.get_dummies(meds,columns=['typevalue'])


In [68]:
meds_dummies['hour'] = meds_dummies.startdate.apply(split_hour)


In [69]:
meds_dummies

,uhid,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,typevalue_Inotropes,hour
0,RSHI.0000016373,2019-02-19 17:51:20.359000+08:00,2019-02-19 17:51:20.359000+08:00,Colisitin,TYPE0001,1,0,2019-02-19 17
1,RSHI.0000016373,2019-02-19 17:51:20.359000+08:00,2019-02-19 17:51:20.359000+08:00,Meropenem,TYPE0001,1,0,2019-02-19 17
2,RSHI.0000016373,2019-02-18 13:13:01.299000+08:00,2019-02-18 13:13:01.299000+08:00,Piperacilin Tazobactam,TYPE0001,1,0,2019-02-18 13
3,RSHI.0000016373,2019-02-18 13:13:01.299000+08:00,2019-02-18 13:13:01.299000+08:00,Amikacin,TYPE0001,1,0,2019-02-18 13
4,RSHI.0000016373,2019-02-19 21:41:28.752000+08:00,2019-02-19 21:41:28.752000+08:00,Vancomycin,TYPE0001,1,0,2019-02-19 21
5,RSHI.0000016373,2019-02-19 18:02:15.503000+08:00,2019-02-19 18:02:15.503000+08:00,Dopamine,TYPE0004,0,1,2019-02-19 18
6,RSHI.0000016373,2019-02-19 18:02:15.503000+08:00,2019-02-19 18:02:15.503000+08:00,Epinephrine,TYPE0004,0,1,2019-02-19 18
7,RSHI.0000016373,2019-02-20 00:59:10.504000+08:00,2019-02-20 00:59:10.504000+08:00,Dobutamine,TYPE0004,0,1,2019-02-20 00


In [70]:
meds_dummies = pd.get_dummies(meds,columns=['typevalue'])
meds_dummies['hour'] = meds_dummies.startdate.apply(split_hour)
s2.drop(['hour_x','hour_y'],axis=1,inplace=True)

In [71]:
s2['hour'] = s2.hour_series.apply(split_hour)

In [72]:
meds_dummies

,uhid,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,typevalue_Inotropes,hour
0,RSHI.0000016373,2019-02-19 17:51:20.359000+08:00,2019-02-19 17:51:20.359000+08:00,Colisitin,TYPE0001,1,0,2019-02-19 17
1,RSHI.0000016373,2019-02-19 17:51:20.359000+08:00,2019-02-19 17:51:20.359000+08:00,Meropenem,TYPE0001,1,0,2019-02-19 17
2,RSHI.0000016373,2019-02-18 13:13:01.299000+08:00,2019-02-18 13:13:01.299000+08:00,Piperacilin Tazobactam,TYPE0001,1,0,2019-02-18 13
3,RSHI.0000016373,2019-02-18 13:13:01.299000+08:00,2019-02-18 13:13:01.299000+08:00,Amikacin,TYPE0001,1,0,2019-02-18 13
4,RSHI.0000016373,2019-02-19 21:41:28.752000+08:00,2019-02-19 21:41:28.752000+08:00,Vancomycin,TYPE0001,1,0,2019-02-19 21
5,RSHI.0000016373,2019-02-19 18:02:15.503000+08:00,2019-02-19 18:02:15.503000+08:00,Dopamine,TYPE0004,0,1,2019-02-19 18
6,RSHI.0000016373,2019-02-19 18:02:15.503000+08:00,2019-02-19 18:02:15.503000+08:00,Epinephrine,TYPE0004,0,1,2019-02-19 18
7,RSHI.0000016373,2019-02-20 00:59:10.504000+08:00,2019-02-20 00:59:10.504000+08:00,Dobutamine,TYPE0004,0,1,2019-02-20 00


In [73]:
s2

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,rbs,skintemp,centraltemp,temp,day,visitdate,visittime,currentdateweight,currentdateheight,hour
0,0,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,None,NaN,NaN,2019-02-17,2019-02-17,17:18:24.308573,2520.0,40.5,2019-02-17 14
1,1,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,None,NaN,NaN,2019-02-17,2019-02-17,17:18:24.308573,2520.0,40.5,2019-02-17 14
2,2,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,None,NaN,NaN,2019-02-17,2019-02-17,17:18:24.308573,2520.0,40.5,2019-02-17 14
3,3,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,None,NaN,NaN,2019-02-17,2019-02-17,17:18:24.308573,2520.0,40.5,2019-02-17 14
4,4,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,None,NaN,NaN,2019-02-17,2019-02-17,17:18:24.308573,2520.0,40.5,2019-02-17 14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4093,4093,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,None,36.5,36.5,2019-02-20,2019-02-20,07:00:00,2811.0,NaN,2019-02-20 11
4094,4094,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,None,36.5,36.5,2019-02-20,2019-02-20,07:00:00,2811.0,NaN,2019-02-20 11
4095,4095,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,None,36.5,36.5,2019-02-20,2019-02-20,07:00:00,2811.0,NaN,2019-02-20 11
4096,4096,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,NaN,None,36.5,36.5,2019-02-20,2019-02-20,07:00:00,2811.0,NaN,2019-02-20 11


In [74]:
s3 = pd.merge(s2,meds_dummies,on = ['uhid','hour'], how='left')

In [75]:
s3['day'] = s3['hour_series'].apply(split_date_1)

In [76]:
s3

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,visittime,currentdateweight,currentdateheight,hour,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,typevalue_Inotropes
0,0,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,17:18:24.308573,2520.0,40.5,2019-02-17 14,NaT,NaT,NaN,NaN,NaN,NaN
1,1,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,17:18:24.308573,2520.0,40.5,2019-02-17 14,NaT,NaT,NaN,NaN,NaN,NaN
2,2,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,17:18:24.308573,2520.0,40.5,2019-02-17 14,NaT,NaT,NaN,NaN,NaN,NaN
3,3,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,17:18:24.308573,2520.0,40.5,2019-02-17 14,NaT,NaT,NaN,NaN,NaN,NaN
4,4,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,17:18:24.308573,2520.0,40.5,2019-02-17 14,NaT,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4273,4093,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,07:00:00,2811.0,NaN,2019-02-20 11,NaT,NaT,NaN,NaN,NaN,NaN
4274,4094,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,07:00:00,2811.0,NaN,2019-02-20 11,NaT,NaT,NaN,NaN,NaN,NaN
4275,4095,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,07:00:00,2811.0,NaN,2019-02-20 11,NaT,NaT,NaN,NaN,NaN,NaN
4276,4096,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,07:00:00,2811.0,NaN,2019-02-20 11,NaT,NaT,NaN,NaN,NaN,NaN


In [80]:
s3

,Unnamed: 0,actual_DOB,uhid,dischargeddate,actual_DOA,gender,birthweight,birthlength,birthheadcircumference,inout_patient_status,...,visittime,currentdateweight,currentdateheight,hour,startdate,medicineorderdate,medicinename,medicationtype,typevalue_Antibiotics,typevalue_Inotropes
0,0,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,17:18:24.308573,2520.0,40.5,2019-02-17 14,NaT,NaT,NaN,NaN,NaN,NaN
1,1,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,17:18:24.308573,2520.0,40.5,2019-02-17 14,NaT,NaT,NaN,NaN,NaN,NaN
2,2,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,17:18:24.308573,2520.0,40.5,2019-02-17 14,NaT,NaT,NaN,NaN,NaN,NaN
3,3,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,17:18:24.308573,2520.0,40.5,2019-02-17 14,NaT,NaT,NaN,NaN,NaN,NaN
4,4,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,17:18:24.308573,2520.0,40.5,2019-02-17 14,NaT,NaT,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4273,4093,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,07:00:00,2811.0,NaN,2019-02-20 11,NaT,NaT,NaN,NaN,NaN,NaN
4274,4094,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,07:00:00,2811.0,NaN,2019-02-20 11,NaT,NaT,NaN,NaN,NaN,NaN
4275,4095,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,07:00:00,2811.0,NaN,2019-02-20 11,NaT,NaT,NaN,NaN,NaN,NaN
4276,4096,2019-02-17 14:54:00.000,RSHI.0000016373,2019-02-20 11:11:32.951,2019-02-17 14:54:00.000,Male,2520.0,40.5,33.0,In Born,...,07:00:00,2811.0,NaN,2019-02-20 11,NaT,NaT,NaN,NaN,NaN,NaN


In [77]:
#PN
cur8= con.cursor()
cur8.execute("SELECT t1.uhid,t1.entrydatetime,t1.totalparenteralvolume,t1.total_intake FROM apollo.babyfeed_detail AS t1 where t1.uhid = '"+uhidPatient+"'")
cols8 = list(map(lambda x: x[0], cur8.description))
pn = pd.DataFrame(cur8.fetchall(),columns=cols8)

In [78]:
pn

,uhid,entrydatetime,totalparenteralvolume,total_intake
0,RSHI.0000016373,2019-02-18 22:48:30.536005+08:00,202.64,202.64
1,RSHI.0000016373,2019-02-19 01:30:04.544501+08:00,256.50,256.5
2,RSHI.0000016373,2019-02-18 01:22:21.575613+08:00,NaN,151.2
3,RSHI.0000016373,2019-02-18 01:23:04.427059+08:00,201.60,201.6
4,RSHI.0000016373,2019-02-18 01:23:23.438760+08:00,201.60,201.6
5,RSHI.0000016373,2019-02-19 12:41:31.430741+08:00,256.50,256.5
6,RSHI.0000016373,2019-02-19 01:34:16.637890+08:00,256.50,256.5
7,RSHI.0000016373,2019-02-19 21:27:46.848212+08:00,154.98,154.98
8,RSHI.0000016373,2019-02-19 18:18:47.941370+08:00,206.64,206.64


In [79]:
pn['hour'] = pn.entrydatetime.apply(split_hour)
pn['total_intake'] = pn['total_intake'].astype(float)
pn['tpn-tfl'] = pn['totalparenteralvolume']/pn['total_intake']

In [ ]:
s4 = pd.merge(s3,pn,on = ['uhid','hour'], how='left')

In [ ]:
s4

In [ ]:
len(s4.uhid.unique())

In [ ]:
s4.drop_duplicates(subset=['uhid','hour_series'],inplace=True)

In [ ]:
q = pd.DataFrame()

In [ ]:
for i in s4.uhid.unique():
    x = s4[s4['uhid']==i]
    x['currentdateheight'].fillna(method='ffill',inplace=True)
    x['currentdateweight'].fillna(method='ffill',inplace=True)
    q = q.append(x,ignore_index=True)

In [ ]:
def abd(x):
    
    try:   
        
        if x>200:
            return x/10
        else:
            return x
    except:
        pass
    

In [ ]:
q['abdomen_girth'] = pd.to_numeric(q['abdomen_girth'], errors='coerce')
q['abdomen_girth'] = q['abdomen_girth'].apply(abd)

In [ ]:
dt = pd.DataFrame()
for i in q.uhid.unique():
    x = q[q['uhid']==i]
    #for j in y['day'].unique():
    #x = y[y['day']==j]
    x['abdomen_girth'].fillna(method='bfill',inplace=True)
    x['currentdateweight'].fillna(method='bfill',inplace=True)
    x['currentdateheight'].fillna(method='bfill',inplace=True)
    x['totalparenteralvolume'].fillna(method='ffill',inplace=True)
    x['total_intake'].fillna(method='ffill',inplace=True)
    x['tpn-tfl'].fillna(method='ffill',inplace=True)
    dt = dt.append(x,ignore_index=True)

In [ ]:
dt.urine_per_hour.max()

In [ ]:
dt.urine_per_hour.unique()

In [ ]:
a = dt.replace([np.inf, -np.inf], np.nan)

In [ ]:
def abd_2(x):
    if not(x is None) and x < 0:
        return ((-1)*x)/10
    elif not(x is None) and x > 200:
        return x/10
    else:
        return x
    
a['abdomen_girth'] = a['abdomen_girth'].apply(abd_2)

def abd_3(x):
    if not(x is None) and x < 15:
        return np.nan
    elif not(x is None) and x > 50:
        return np.nan
    else:
        return x
    
a['abdomen_girth'] = a['abdomen_girth'].apply(abd_3)

def upm(x):
    if not(x is None) and x > 100:
        return np.nan
    else:
        return x
    
a['urine_per_hour'] = a['urine_per_hour'].apply(upm)

def temp_f_c(x):
    if not(x is None) and x > 90:
        return (x - 32)*(5/9)
    else:
        return x
    
a['temp'] = a['temp'].apply(temp_f_c)

def weight_correct(x):
    if not(x is None) and x<10:
        return x*100
    elif x>5000:
        return x/10
    else:
        return x
    
def weight_correct_2(x):
    if not(x is None) and x<300:
        return x*10
    elif x>5000:
        return x/10
    else:
        return x
    
a.currentdate = a.currentdateweight.apply(weight_correct)
a.currentdate = a.currentdateweight.apply(weight_correct_2)

def height_correct(x):
    if not(x is None) and x < 10:
        return np.nan
    else:
        return x
    
def rbs_correct(x):
    if not(x is None) and x > 700:
        return np.nan
    else:
        return x
    
a['currentdateheight'] = a['currentdateheight'].apply(height_correct)

a['rbs'] = a['rbs'].apply(rbs_correct)



In [ ]:
len(a.uhid.unique())

In [ ]:
data = a.copy()

In [ ]:
def split_date_1(x):
    try:
        t=str(x)
        return str(t.split(" ")[0])
    except:
        pass

In [ ]:
data['day'] = data['hour_series'].apply(split_date_1)

In [ ]:
def to_date(x):
    return pd.to_datetime(x)

In [ ]:
data['hour_series'] = data['hour_series'].apply(to_date)

In [ ]:
len(data.uhid.unique())

In [ ]:
data.to_csv(caseType+seperator+uhidPatient+seperator+"intermittent_checkpoint_2.csv")

In [ ]:
start_date = pd.to_datetime(x['day'].iloc[0]+" " + "08:00:00") - timedelta(hours=24)

In [ ]:
x['stool_passed'].dropna()

In [ ]:
df = pd.DataFrame()
for i in data.uhid.unique():
    x = data[data['uhid']==i]
    x.rbs.fillna(method='ffill',inplace=True)
    x.currentdateweight.fillna(method='ffill',inplace=True)
    x.currentdateheight.fillna(method='ffill',inplace=True)
    n = math.ceil(len(x)/24)+1
    start_date = pd.to_datetime(x['day'].iloc[0]+" " + "08:00:00") - timedelta(hours=24)
    for i in range(int(n)):
        
       
        y = x[(x['hour_series']>=start_date + timedelta(hours=24*i)) & (x['hour_series']<=start_date + timedelta(hours=24*(i+1)))]
        y['stool_day_total'] = (y['stool_passed'].sum())/60
        df =df.append(y,ignore_index=True)


In [ ]:
y

In [ ]:
df.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_3.csv')

In [ ]:
test = df.copy()

In [ ]:
dq = pd.DataFrame()

In [ ]:
for i in test.uhid.unique():
    x = test[test['uhid']==i]
    x.abdomen_girth.fillna(method='ffill',inplace=True)
    dq = dq.append(x, ignore_index=True)

In [ ]:
final = pd.DataFrame()

In [ ]:
for i in dq.uhid.unique():
    x = dq[dq['uhid']==i]
    x['abd_difference_y'] = x['abdomen_girth'] - x['abdomen_girth'].shift(periods=1)
    final = final.append(x,ignore_index=True)

In [ ]:
final.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_4.csv')

In [ ]:
cur1 = con.cursor()
cur1.execute("SELECT t1.uhid,t1.starttime,t1.pulserate, t1.ecg_resprate, t1.spo2, t1.heartrate, t2.dischargestatus,t1.mean_bp,t1.sys_bp,t1.dia_bp  FROM apollo.device_monitor_detail_dump AS t1 LEFT JOIN apollo.baby_detail AS t2 ON t1.uhid=t2.uhid WHERE (t2.dischargestatus = '"+caseType+"' AND t2.dateofadmission > '2018-07-01' and t2.uhid = '"+ uhidPatient+"') OR (t2.dischargestatus = '"+caseType+"' AND t2.dateofadmission > '2018-07-01' and t2.uhid = '"+ uhidPatient+"');")
cols1 = list(map(lambda x: x[0], cur1.description))
ds = pd.DataFrame(cur1.fetchall(),columns=cols1)
ds.sort_values('starttime',inplace=True)

In [ ]:
ds

In [ ]:
ds[(ds.mean_bp=='94') | (ds.mean_bp=='71') | (ds.mean_bp=='59')]

In [ ]:
test = ds.drop_duplicates(subset=['uhid','starttime'],keep='first')

In [ ]:
cur_vent = con.cursor()
cur_vent.execute("SELECT t1.uhid,t1.start_time,t1.creationtime,t1.peep, t1.pip,t1.map ,t1.tidalvol, t1.minvol,t1.ti,t1.fio2 FROM apollo.device_ventilator_detail_dump AS t1 LEFT JOIN apollo.baby_detail AS t2 ON t1.uhid=t2.uhid WHERE (t2.dischargestatus = 'Discharge' OR t2.dischargestatus = '"+caseType+"') and t2.uhid = '"+uhidPatient+"';")
cols_vent = list(map(lambda x: x[0], cur_vent.description))
ventilator_cont = pd.DataFrame(cur_vent.fetchall(),columns=cols_vent)

test_vent = ventilator_cont.drop_duplicates(subset=['uhid','start_time'],keep='first')

def split_date(x):
    try:
        t=str(x)
        return t.split(":")[0] + ":" + t.split(":")[1]
    except:
        pass

test['date'] = test.starttime.apply(split_date)
test_vent['date'] = test_vent.start_time.apply(split_date)

In [ ]:
cont_data = pd.merge(test,test_vent,on=['uhid','date'],how='left',copy=False)
test_cont = cont_data.drop_duplicates(subset=['uhid','starttime','heartrate'],keep='first')


In [ ]:
test_cont['hour_series'] = test_cont['date'].apply(split_hour)

In [ ]:
def to_str(x):
    return str(x)

In [ ]:
test_cont['ref_hour'] = test_cont['hour_series'].apply(to_str)
final['ref_hour'] = final['hour_series'].apply(to_str)

In [ ]:
def con_time(x):
    return str(x).split("+")[0].split(":")[0] + ":" +str(x).split("+")[0].split(":")[1]

In [ ]:
test_cont['cont_time'] = test_cont.starttime.apply(con_time)

In [ ]:
def con_time_2(x):
    return str(x).split(":")[0] +":"+str(x).split(":")[1]

In [ ]:
final['cont_time'] = final.ref_hour.apply(con_time_2)

In [ ]:
qw = pd.merge(final,test_cont, on=['uhid','cont_time'],how='left')

In [ ]:
#final[final['cont_time']=='2019-09-30 11:18']

In [ ]:
#qw.dropna(subset=['series'],inplace=True)

In [ ]:
qw.to_csv(caseType+seperator+uhidPatient+seperator+'intermediate_checkpoint_new_5.csv')

In [ ]:
qw.count()